# Notebook for Evan - Generating Satellite Releases
Code Author: Sahar El Abbadi
Date: 2023-04-04


In [17]:
# Setup

# imports
import pandas as pd
import datetime
import pathlib
from datetime import timedelta
from methods_source import calc_average_release

## Satellite release dataframe
Create satellite release summary file with timestamps as datetime objects in UTC. The start and stop times of the relases will be used as inputs to the calc_average_releases function.

Dummy dataset includes one example time period when we have no release record (e.g. satellite release occurred on a weekend, and there is no meter file for the date in question), and one release where we have a metered data file for the time period in question.

Import satellite data

In [105]:
# overpass_filename = pathlib.PurePath('04_figures', 'parity_plots', fig_name)
# pathlib.PurePath('01_clean_reports', 'Satellite overpass datetime 20230404.xlsx')

satellite_overpass_times = pd.read_excel(pathlib.PurePath('00_raw_reports', 'Satellite_results_consolidated_20230405e.xlsx'), )
# satellite_overpass_times = pd.read_excel(pathlib.PurePath('01_clean_reports', 'Satellite_results_consolidated_20230404b.xlsx'), )
satellite_overpass_times['end_release'] = pd.to_datetime(satellite_overpass_times['DateTime (UTC)'])
satellite_overpass_times['start_release'] = satellite_overpass_times['end_release'] - timedelta(minutes=5)
satellite_overpass_times
# satellite_timestamps = satellite_overpass_times
# satellite_timestamps

,Team,Satellite,PerformerExperimentID,FacilityID,EquipmentUnitID,DateOfSurvey,Timestamp (hyperspectral technologies only),StartTime,EndTime,SurveyTime,...,WindSpeedUncertaintyType,WindDirection,NumberOfEmissionSourcesReported,Notes,Data source,QC,QC_lax,DateTime (UTC),end_release,start_release
0,GHGSat,Sentinel-2,94e765d9-f8fd-49b0-9851-57cbd7dcb4b6 - 10/26/2022,1,1.0,2022-10-18 17:49:00,17:49:00,NaN,NaN,NaN,...,Excluded,Excluded,NaN,Release cancelled by Stanford due to system ma...,NaN,SE,OK,10/18/2022 17:49:00,2022-10-18 17:49:00,2022-10-18 17:44:00
1,GHGSat,Sentinel-2,94e765d9-f8fd-49b0-9851-57cbd7dcb4b6 - 10/26/2022,1,1.0,2022-10-26 18:14:48,2022-10-26 18:14:48,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,DataReportingTemplate_AerialAppendix_GHGSat_S2...,OK,OK,10/26/2022 18:14:48,2022-10-26 18:14:48,2022-10-26 18:09:48
2,GHGSat,Sentinel-2,94e765d9-f8fd-49b0-9851-57cbd7dcb4b6 - 10/26/2022,1,1.0,2022-11-05 18:14:40,18:14:40,NaN,NaN,NaN,...,Excluded,Excluded,NaN,Release cancelled by Stanford due to personnel...,NaN,SE,SE,11/05/2022 18:14:40,2022-11-05 18:14:40,2022-11-05 18:09:40
3,GHGSat,Sentinel-2,94e765d9-f8fd-49b0-9851-57cbd7dcb4b6 – 11/08/2022,1,1.0,2022-11-08 18:24:30,2022-11-08 18:24:30,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,DataReportingTemplate_AerialAppendix_GHGSat_S2...,OK,OK,11/08/2022 18:24:30,2022-11-08 18:24:30,2022-11-08 18:19:30
4,GHGSat,Sentinel-2,94e765d9-f8fd-49b0-9851-57cbd7dcb4b6 – 11/15/2022,1,1.0,2022-11-15 18:14:28,2022-11-15 18:14:28,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,DataReportingTemplate_AerialAppendix_GHGSat_S2...,OK,OK,11/15/2022 18:14:28,2022-11-15 18:14:28,2022-11-15 18:09:28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198,Orbio,Sentinel-2,2,1,1.0,2022-11-08 00:00:00,18:25:05,NaN,NaN,NaN,...,NaN,218.700626,NaN,NaN,2022_OrbioEarth_satellite_report.xlsx,OK,OK,11/08/2022 18:25:05,2022-11-08 18:25:05,2022-11-08 18:20:05
199,Orbio,Sentinel-2,3,1,1.0,2022-11-15 00:00:00,18:15:08,NaN,NaN,NaN,...,NaN,312.276679,NaN,NaN,2022_OrbioEarth_satellite_report.xlsx,OK,OK,11/15/2022 18:15:08,2022-11-15 18:15:08,2022-11-15 18:10:08
200,Orbio,Sentinel-2,4,1,1.0,2022-11-18 00:00:00,18:25:03,NaN,NaN,NaN,...,NaN,257.957679,NaN,NaN,2022_OrbioEarth_satellite_report.xlsx,OK,OK,11/18/2022 18:25:03,2022-11-18 18:25:03,2022-11-18 18:20:03
201,Orbio,Sentinel-2,5,1,1.0,2022-11-25 00:00:00,18:15:05,NaN,NaN,NaN,...,NaN,14.320893,NaN,NaN,2022_OrbioEarth_satellite_report.xlsx,OK,OK,11/25/2022 18:15:05,2022-11-25 18:15:05,2022-11-25 18:10:05


## Calculate the gas flow rate during release periods

Use calc_average_releases function to generate gas flow profile for each time period of interest. Here, I apply the function to each row in the satellite_releases dataframe, although other methods may be used as well.

Function inputs:
- start_t: datetime object in UTC
- stop_t: datetime object in UTC

Notes:
- Function will return an error message if start_t and stop_t are on different dates (ie you cannot calculate the average flow rate if they straddle different days of testing)
- Function will return an error message if start_t is chronologically after stop_t
- If start_t and end_t occur on a date for which we do not have metered data, function will output values for a zero release (flow rate of 0 kgh, uncertainty of 0, meter and gas composition of np.nan)

Function outputs a Python dictionary with the following keys:
- gas_kgh_mean: mean gas flow rate over the time period of interest, as whole gas
- gas_kgh_sigma: standard deviation of the meter reading over the period of interest, as whole gas. This value represents the physical variability in the flow rate.
- meter_sigma: the standard deviation calculated based on the uncertainty in the meter reading for the flow rate during the period in question. Emerson reports uncertainty as a percentage of flow rate, and the function converts this value to a sigma value with units of kgh whole gas
- ch4_fraction_{gas_comp_source}: the fraction methane based on the input source for gas composition. Gas composition can be 'su_normalized', 'su_raw', or 'km'. See other documentation for additional details. This value is averaged over the time period of interest, although unless the trailer was changed mid-release, value is expected to be constant
- ch4_fraction{gas_comp_source}_sigma: the sigma value associated with the gas composition, representative of the uncertainty associated with measurements of methane mole fraction.
- ch4_kgh_mean: mean methane flow rate, calculated from the gas flow rate and methane mole fraction
- cg4_kgh_sigma: total uncertainty in the methane flow rate. This value combines physical variability in gas flow rate (gas_kgh_sigma) with uncertainty in the meter reading (meter_sigma) and uncertainty in gas composition (ch4_fraction_{gas_comp_source}_sigma.

In [106]:
# Run function to determine flow rates for each release

release_rates =[]

for jj in satellite_overpass_times.index:
    start_t = satellite_overpass_times.loc[jj, 'start_release']
    end_t = satellite_overpass_times.loc[jj, 'end_release']
    release_details = calc_average_release(start_t, end_t)
    release_rates.append(release_details)

final_releases = pd.DataFrame(release_rates)
summary = pd.concat([satellite_overpass_times, final_releases], axis=1)

# print(summary)
# summary.to_csv(pathlib.PurePath('01_clean_reports', 'Satellite_results_consolidated_clean_20230405.csv'))
summary.to_csv(pathlib.PurePath('05_satellites', '01_input_data', 'Satellite_results_consolidated_clean_20230405f.csv'))


# Scratch pad

In [78]:
satellite_overpass_times.loc[68, ]

Team                                                          LARS
Satellite                                                    EnMap
PerformerExperimentID                                            3
FacilityID                                                       1
EquipmentUnitID                                                1.0
DateOfSurvey                                   2022-11-13 00:00:00
Timestamp (hyperspectral technologies only)               19:02:08
StartTime                                                      NaN
EndTime                                                        NaN
SurveyTime                                                     NaN
Gas                                                        Methane
FacilityEmissionRate                                      Excluded
FacilityEmissionRateUpper                                 Excluded
FacilityEmissionRateLower                                 Excluded
UncertaintyType                                           Excl

In [93]:
# Dummy dataframe to be filled with satellite data
# Use UTC for start and end timestamps
satellite_timestamps = pd.DataFrame({
    'release_id': [1, 2, 3],
    'satellite': ['Test1', 'Test2', 'Test3'],
    'team_analyzing': ['Luis', 'Maxar', 'Kayrros'],
    'start_release':[datetime.datetime(2022, 10, 15, 18, 16, 10), datetime.datetime(2022, 10, 25, 17, 44, 22), datetime.datetime(2022, 10, 25, 17, 44, 22)],
    'end_release':[datetime.datetime(2022, 10, 15, 18, 36, 00), datetime.datetime(2022, 10, 25, 18, 10, 00), datetime.datetime(2022, 10, 25, 18, 10, 00)],
})

In [94]:
# Run function to determine flow rates for each release

release_rates =[]

for index, release in satellite_timestamps.iterrows():
    start_t = release.start_release
    end_t = release.end_release
    release_details = calc_average_release(start_t, end_t)
    release_rates.append(release_details)

final_releases = pd.DataFrame(release_rates)
summary = pd.concat([satellite_timestamps, final_releases], axis=1)

print(summary)

   release_id satellite team_analyzing       start_release  \
0           1     Test1           Luis 2022-10-15 18:16:10   
1           2     Test2          Maxar 2022-10-25 17:44:22   
2           3     Test3        Kayrros 2022-10-25 17:44:22   

          end_release  gas_kgh_mean  gas_kgh_sigma  meter_sigma  \
0 2022-10-15 18:36:00      0.000000       0.000000          NaN   
1 2022-10-25 18:10:00   1025.589677      99.868814     1.310053   
2 2022-10-25 18:10:00   1025.589677      99.868814     1.310053   

   ch4_fraction_km  ch4_fraction_km_sigma  ch4_kgh_mean  ch4_kgh_sigma  
0              NaN                    NaN           NaN            NaN  
1         0.950644               0.001366    974.970673      94.958189  
2         0.950644               0.001366    974.970673      94.958189  


In [57]:
start_t, end_t
# calc_average_release(start_t, end_t)

(Timestamp('2022-10-25 17:44:22'), Timestamp('2022-10-25 18:10:00'))

In [81]:
jj = 69
start_t = satellite_overpass_times.loc[jj, 'start_release']
end_t = satellite_overpass_times.loc[jj, 'end_release']
start_t, end_t

(Timestamp('2022-11-15 23:55:00'), Timestamp('2022-11-16 00:00:00'))

In [82]:
# satellite_overpass_times
calc_average_release(start_t, end_t)
# release_details = calc_average_release(start_t, end_t)
# release_details

Do not attempt to calculate average flow across multiple dates. Please consider a new start or end time.
